In [1]:
import os

In [2]:
%pwd

'D:\\Desktop\\Superteams AI\\Task 1 DSPy-MiniLM_12--Qdrant_RAG_with_Prometheus_Grafana\\RAG-Stack-using-DSPy-Qdrant-Mistral-7B\\Research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'D:\\Desktop\\Superteams AI\\Task 1 DSPy-MiniLM_12--Qdrant_RAG_with_Prometheus_Grafana\\RAG-Stack-using-DSPy-Qdrant-Mistral-7B'

In [5]:
import logging
from pathlib import Path

In [6]:
# Configure the logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [7]:
import pyarrow.parquet as pq

# Read the Parquet file into a pyarrow Table
table = pq.read_table("Dataset/train-00000-of-00001-a77e2814210655f1.parquet")

# Convert the Table to a pandas DataFrame
df = table.to_pandas()

# Now you can work with the DataFrame as usual
df.head()


,instruction,input,output,__index_level_0__
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",137213
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,67540
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,109854
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,137309
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...",135382


In [8]:
# Create dfcopy with 100 rows
dfcopy = df.head(15).copy()

# Display the first few rows
dfcopy.head()

,instruction,input,output,__index_level_0__
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",137213
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,67540
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,109854
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,137309
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...",135382


In [9]:
dfcopy = dfcopy.drop("__index_level_0__", axis=1)
dfcopy.head()

,instruction,input,output
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in..."
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ..."


In [10]:
# 'sentence-transformers/all-MiniLM-L6-v2'

In [11]:
# from transformers import AutoTokenizer, AutoModel
# from pathlib import Path

# def download_model_and_tokenizer(model_name, save_path):
#     """
#     Download and save both the model and the tokenizer to the specified directory.

#     Parameters:
#         model_name (str): Name of the model to download.
#         save_path (str or Path): Path to the directory where the model and tokenizer will be saved.
#     """
#     # Create the save path if it doesn't exist
#     save_path = Path(save_path)
#     save_path.mkdir(parents=True, exist_ok=True)
    
#     # Initialize tokenizer and model
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModel.from_pretrained(model_name)
    
#     # Save tokenizer
#     tokenizer.save_pretrained(save_path)
    
#     # Save model
#     model.save_pretrained(save_path)

# # Example usage
# model_name = 'sentence-transformers/all-MiniLM-L6-v2'  # Model name to download
# save_path = Path("MiniLM-L6-v2/")  # Path where model and tokenizer will be saved
# download_model_and_tokenizer(model_name, save_path)


In [41]:
from transformers import AutoTokenizer, AutoModel

def load_model_and_tokenizer(model_path):
    """
    Load the model and tokenizer from the specified directory.

    Parameters:
        model_path (str or Path): Path to the directory containing the saved model and tokenizer.

    Returns:
        tokenizer (transformers.PreTrainedTokenizer): Loaded tokenizer.
        model (transformers.PreTrainedModel): Loaded model.
    """
    model_path = Path(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
    return tokenizer, model

# Load the model and tokenizer
model_path = Path("MiniLM-L6-v2/")
tokenizer, model = load_model_and_tokenizer(model_path)

In [78]:
def generate_embedding(text):
    encoder = EncoderCT2fromHfHub(
        model_name_or_path="michaelfeil/ct2fast-all-MiniLM-L6-v2",
        device="cuda",
        compute_type="int8_float16"
    )
    # Generate embeddings
    embeddings = encoder.generate([text])
    # Process embeddings here, e.g., extract pooler_output, convert to list, etc.
    # This step depends on the structure of the embeddings returned by your model
    return embeddings

In [79]:
 generate_embedding("Azithromycin is an antibiotic that is commonly used to treat bacterial and viral infections in the respiratory system, such as strep throat and pneumonia. It works by inhibiting the growth of certain bacteria and viruses that cause these infections by interfering with their ability to reproduce and replicate their DNA. Additionally, azithromycin has been shown to have antiviral effects against certain viruses, including SARS-CoV-2, the virus that causes COVID-19.")


{'pooler_output': tensor([[-7.3051e-03,  5.3436e-02,  4.9286e-02,  5.5756e-02, -1.4343e-02,
           9.7809e-03, -1.6129e-02,  2.8091e-02,  1.0979e-02,  1.2684e-04,
           4.6959e-03,  9.0332e-02,  6.7139e-02,  5.2551e-02, -6.4087e-02,
           1.2195e-01,  1.2549e-01, -1.0963e-02, -1.5678e-03,  4.9500e-02,
          -1.7120e-02, -1.5045e-02,  8.3740e-02,  3.2562e-02,  1.2398e-02,
           6.8481e-02,  9.2087e-03,  1.3657e-02, -4.3854e-02,  8.2458e-02,
           8.2764e-02,  5.1910e-02, -1.1688e-01,  6.7932e-02, -1.6174e-02,
          -1.4197e-01,  4.5837e-02,  5.0018e-02, -3.7270e-03, -1.5230e-03,
           3.8116e-02,  1.4549e-02, -6.0211e-02,  1.5930e-01, -2.3560e-02,
           3.7231e-02, -1.7175e-01,  1.6785e-02,  8.1726e-02, -7.0915e-03,
          -9.3079e-02,  4.4128e-02,  4.9652e-02,  1.4313e-02, -4.8065e-02,
           7.5867e-02,  1.0818e-02, -1.6708e-02,  6.3965e-02,  1.0901e-01,
           4.2114e-02, -4.6234e-03, -1.2848e-02,  4.7333e-02, -6.3171e-02,
        

In [51]:
from hf_hub_ctranslate2 import EncoderCT2fromHfHub
import pandas as pd

# Function to initialize the encoder and generate embeddings
def generate_embeddings(text):
    encoder = EncoderCT2fromHfHub(
        model_name_or_path="michaelfeil/ct2fast-all-MiniLM-L6-v2",
        device="cuda",
        compute_type="int8_float16"
    )
    # Generate embeddings
    embeddings = encoder.generate([text])
    # Process embeddings here, e.g., extract pooler_output, convert to list, etc.
    # This step depends on the structure of the embeddings returned by your model
    processed_embeddings = embeddings['pooler_output'].detach().cpu().numpy().flatten().tolist()
    return processed_embeddings

def apply_embeddings(dataframe, input_column, output_column):
    dataframe[output_column] = dataframe[input_column].apply(generate_embeddings)
    return dataframe

# Usage

dfcopy = apply_embeddings(dfcopy, 'output', 'embeddings')
dfcopy.head()


,instruction,input,output,embeddings
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...","[0.05584716796875, 0.01099395751953125, 0.0569..."
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,"[-0.03765869140625, 0.0582275390625, -0.002435..."
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,"[0.0408935546875, 0.12445068359375, 0.00955963..."
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,"[0.03314208984375, 0.00809478759765625, 0.0678..."
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...","[-0.004528045654296875, -0.0316162109375, 0.04..."


In [36]:
# from hf_hub_ctranslate2 import EncoderCT2fromHfHub

# # Initialize the encoder with the model from Hugging Face Hub
# model_name_2 = "michaelfeil/ct2fast-all-MiniLM-L6-v2"
# encoder = EncoderCT2fromHfHub(
#     model_name_or_path=model_name_2, 
#     device="cuda", 
#     compute_type="int8_float16"
# )

# # Encode the text sequences
# # Note: The actual method to call and its parameters might differ based on the specific encoder implementation.
# # This is a generic way to process input assuming the encoder expects tokenized text.
# inputs = ["I like soccer", "I like tennis", "The Eiffel Tower is in Paris"]
# encoded_outputs = encoder.generate(inputs)

# # Process the encoded outputs
# print(encoded_outputs)


In [16]:
df.head()

,instruction,input,output,__index_level_0__
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",137213
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,67540
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,109854
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,137309
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...",135382


# Qdrant no dspy

In [82]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# Initialize Qdrant client
qdrant_client = QdrantClient(host='localhost', port=6333)
collection_name =  "medical_dataset_noembedding" 

# Specify the vectors' configuration
vectors_config = VectorParams(
    size=model.config.hidden_size,  # The size of your embeddings
    distance=Distance.COSINE  # The distance metric for the vector space
)

# Create or recreate the collection with the specified configuration
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=vectors_config,
    # Optionally, you can specify other parameters for the collection
)



INFO:httpx:HTTP Request: DELETE http://localhost:6333/collections/medical_dataset_noembedding "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset_noembedding "HTTP/1.1 200 OK"


True

In [83]:
# Insert data into Qdrant
for index, row in dfcopy.iterrows():
    # Ensure the embeddings are a flat list of floats
    
    qdrant_client.upsert(
        collection_name=collection_name,
        points=[{
            "id": index,  # Using the dataframe index as the ID
            "vector": row['embeddings'],
            "payload": {
                # "instruction": row['instruction'], 
                # "input": row['input'],
                "output": row['output']
            }
        }]
    )


INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset_noembedding/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset_noembedding/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset_noembedding/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset_noembedding/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset_noembedding/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset_noembedding/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset_noembedding/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset_noembedding/points?wait=true "HTTP/1.1 

In [84]:
logger.info(f"qdrant '{collection_name}' collection created")

INFO:__main__:qdrant 'medical_dataset_noembedding' collection created


In [88]:
from langchain.vectorstores import Qdrant

url = "http://localhost:6333"  # URL where the Qdrant service is running
collection_name =  "medical_dataset_noembedding"  # Name of the collection in Qdrant

# Initialize the Qdrant client with the specified URL
client = QdrantClient(
    url=url,
    prefer_grpc=False  # Indicates whether to use gRPC for communication
)
embedding_model ='sentence-transformers/all-MiniLM-L6-v2'
client.set_model(embedding_model)

logger.info(f"QdrantClient initialized: {client}")  # Prints the client information
logger.info(f"#################################")  # Prints a separator line

# Create a Qdrant object with the specified client, embeddings, and collection name
# Initialize the Qdrant vector store from langchain
db = Qdrant(
    client=client,
    embeddings=dfcopy['embeddings'].tolist(),  # Use the generated embeddings
    collection_name=collection_name
)

logger.info(f"Qdrant vector store initialized: {db}") # Prints the database object information

INFO:__main__:QdrantClient initialized: <qdrant_client.qdrant_client.QdrantClient object at 0x000001F0384A1CC0>
INFO:__main__:#################################
INFO:__main__:Qdrant vector store initialized: <langchain_community.vectorstores.qdrant.Qdrant object at 0x000001F0BA4BBE50>


In [89]:
def similarity_search_with_score(query, k=5):
    query_embedding = generate_embeddings(query)
    search_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=k,
        with_payload=True,
        with_vectors=False
    )
    return search_results

query = "What is Azithromycin?"
search_results = similarity_search_with_score(query=query, k=5)

for result in search_results:
    doc_id = result.id
    score = result.score
    payload = result.payload  # The payload should contain your text or a reference to it.

    # Assuming the payload contains a field 'input' where the text is stored
    doc_content = payload.get('output', 'No content available')

    # Print the similarity score and document content
    logger.info({"score": score, "doc_id": doc_id, "content": doc_content})

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/medical_dataset_noembedding/points/search "HTTP/1.1 200 OK"
INFO:__main__:{'score': 0.7775405, 'doc_id': 1, 'content': 'Hydroxychloroquine vs. Azithromycin for Hospitalized Patients with COVID-19 (HAHPS): Results of a Randomized, Active Comparator Trial'}
INFO:__main__:{'score': 0.73449993, 'doc_id': 11, 'content': 'hello there... gum infections occur because of various factors... herpes infection being one amongst them.... people with herpes infection have a decreased immunity.... acyclovir given would further compromise the status... that is when the infections set in... apart from the immunity, local factors also play an important role in the gum hygiene... people with oral sexual behavior (multiple partners) tend to have more of these infections.... you require oral prophylaxis with appropriate antibiotics... please use betadine rinses (5ml) 3-4 times daily to prevent further bacterial growth.... consult you dental sur

In [90]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp

model_path = r"Mistral7b_Model/mistral-7b-instruct-v0.1.Q5_K_M.gguf"

n_gpu_layers = -1  # Use GPU if possible, rest on CPU
n_batch = 512  # Adjust based on GPU VRAM

try:
    llm = LlamaCpp(
        model_path=model_path,  # Only use model_url, not model_path
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        verbose=True  # Needed for callback manager
    )

    # Add your code using llm here
except Exception as e:
    print(f"Error creating LlamaCpp instance: {e}")

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from Mistral7b_Model/mistral-7b-instruct-v0.1.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:           

In [91]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate.from_template(template)

llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
llm_chain.run(question)


llama_print_timings:        load time =     415.78 ms
llama_print_timings:      sample time =     129.92 ms /   256 runs   (    0.51 ms per token,  1970.40 tokens per second)
llama_print_timings: prompt eval time =     415.57 ms /    46 tokens (    9.03 ms per token,   110.69 tokens per second)
llama_print_timings:        eval time =    7673.29 ms /   255 runs   (   30.09 ms per token,    33.23 tokens per second)
llama_print_timings:       total time =   10088.62 ms /   301 tokens


" \n\n1. The first Super Bowl was in 1967, and Justin Bieber was born on March 1, 2004. Therefore, he has not yet been born when the first Super Bowl took place.\n\n2. Since we are looking for an NFL team that won the Super Bowl during his lifetime, let's focus on years after his birth.\n\n3. The next Super Bowl after his birth occurred on January 29, 2005 between the St. Louis Rams and the Indianapolis Colts. Unfortunately, neither of these teams have won another Super Bowl since then.\n\n4. The next Super Bowl after this one took place on February 5, 2006 between the Seattle Seahawks and the Denver Broncos. Again, none of these teams have won another Super Bowl since then.\n\n5. The next Super Bowl took place on February 4, 2007 between the New England Patriots and the New York Giants. The New England Patriots won that game, making it their third consecutive win and fourth overall. They would go on to win another two Super Bowls before Justin Bieber was born in 2"

In [92]:
from langchain import PromptTemplate, LLMChain


template = '''[INST]: You are a Medical expert analyst bot, below presents a context from which the a question will be asked, give your valuable insights as well.[\INST]\n
Context: {context}.\n
Question: {question}\n
Answer: '''

In [93]:
rag_prompt = PromptTemplate(template=template, input_variables=["context","question"])


callbacks = [StreamingStdOutCallbackHandler()]
llm_chain = LLMChain(prompt=rag_prompt, llm=llm, verbose=True)

In [94]:
def similarity_search_with_score(query, k=5):
    query_embedding = generate_embeddings(query)
    search_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=k,
        with_payload=True,
        with_vectors=False
    )

    # Extract the document content from the payload and include it in the results
    results_with_content = []
    for result in search_results:
        doc_id = result.id
        score = result.score
        payload = result.payload  # The payload should contain your text or a reference to it.
        
        # Extract the document content from the payload
        doc_content = payload.get('output', 'No content available')

        results_with_content.append((score, doc_content))

    # Sort the results based on the similarity score in descending order
    sorted_results = sorted(results_with_content, key=lambda x: x[0], reverse=True)

    # Concatenate the content of the top k results
    concatenated_content = ' '.join([content for _, content in sorted_results[:k]])

    return concatenated_content

# Query for which you want to find similar documents
# query = "What is Azithromycin?"
# concatenated_contents = similarity_search_with_score(query=query, k=2)

# # Print the concatenated content of the top documents
# print("Concatenated content of the top documents:", concatenated_contents)


In [95]:
query = "What is Azithromycin ?"
resp = llm_chain.invoke(
    input={"question":query,
           "context": similarity_search_with_score(query,k=5)
          }
)
print(resp['text'])

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/medical_dataset_noembedding/points/search "HTTP/1.1 200 OK"
Llama.generate: prefix-match hit




> Entering new LLMChain chain...
Prompt after formatting:
[INST]: You are a Medical expert analyst bot, below presents a context from which the a question will be asked, give your valuable insights as well.[\INST]

Context: Hydroxychloroquine vs. Azithromycin for Hospitalized Patients with COVID-19 (HAHPS): Results of a Randomized, Active Comparator Trial hello there... gum infections occur because of various factors... herpes infection being one amongst them.... people with herpes infection have a decreased immunity.... acyclovir given would further compromise the status... that is when the infections set in... apart from the immunity, local factors also play an important role in the gum hygiene... people with oral sexual behavior (multiple partners) tend to have more of these infections.... you require oral prophylaxis with appropriate antibiotics... please use betadine rinses (5ml) 3-4 times daily to prevent further bacterial growth.... consult you dental surgeon for further clini


llama_print_timings:        load time =     415.78 ms
llama_print_timings:      sample time =      62.31 ms /   127 runs   (    0.49 ms per token,  2038.13 tokens per second)
llama_print_timings: prompt eval time =     272.70 ms /   384 tokens (    0.71 ms per token,  1408.16 tokens per second)
llama_print_timings:        eval time =    3659.62 ms /   126 runs   (   29.04 ms per token,    34.43 tokens per second)
llama_print_timings:       total time =    4882.47 ms /   510 tokens



> Finished chain.


Azithromycin is an antibiotic that is commonly used to treat bacterial and viral infections, including pneumonia, sinus infections, and acute bronchitis. It is also used to treat sexually transmitted infections (STIs) such as chlamydia, gonorrhea, and syphilis. In addition, it is sometimes used to treat heart rhythm disorders and inflammatory conditions such as rheumatoid arthritis. It is usually taken orally as a pill or given by injection, and it is generally considered safe


# Dspy

In [96]:
# qdrant_client = QdrantClient(host='localhost', port=6333)

In [97]:
# from langchain.vectorstores import Qdrant

# url = "http://localhost:6333"  # URL where the Qdrant service is running
# collection_name =  "medical_dataset"  # Name of the collection in Qdrant

# # Initialize the Qdrant client with the specified URL
# client = QdrantClient(
#     url=url,
#     prefer_grpc=False  # Indicates whether to use gRPC for communication
# )

# logger.info(f"QdrantClient initialized: {client}")  # Prints the client information
# logger.info(f"#################################")  # Prints a separator line

# # Create a Qdrant object with the specified client, embeddings, and collection name
# # Initialize the Qdrant vector store from langchain
# db = Qdrant(
#     client=client,
#     embeddings=dfcopy['embeddings'].tolist(),  # Use the generated embeddings
#     collection_name=collection_name
# )

# logger.info(f"Qdrant vector store initialized: {db}") # Prints the database object information

In [98]:
import dspy
from dspy.retrieve.qdrant_rm import QdrantRM

from qdrant_client import QdrantClient


url = "http://localhost:6333"  # URL where the Qdrant service is running
collection_name =  "medical_dataset"  # Name of the collection in Qdrant
embedding_model_name = 'sentence-transformers/all-MiniLM-L6-v2'  # Model name
embedding_model_distance = 'Cosine'  # Distance type
qdrant_client = QdrantClient(   url=url,
                                prefer_grpc=False  # Indicates whether to use gRPC for communication
                            )
# Now set the model with the correct parameters
qdrant_client.set_model(embedding_model_name)

qdrant_retriever_model = QdrantRM(collection_name, 
                                  qdrant_client, 
                                  k=3)

dspy.settings.configure(lm=llm, rm=qdrant_retriever_model)

In [99]:
retrieve = dspy.Retrieve(k=3)
question = "What is Azithromycin ?"
topK_passages = retrieve(question).passages

print(f"Top {retrieve.k} passages for question: {question} \n", "\n")

for idx, passage in enumerate(topK_passages):
    print(f"{idx+1}]", passage, "\n")

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/medical_dataset/points/search/batch "HTTP/1.1 400 Bad Request"


UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Wrong input: Vector params for fast-all-minilm-l6-v2 are not specified in config"},"time":0.000084156}'

In [ ]:
import dspy
from dspy.retrieve.qdrant_rm import QdrantRM
from qdrant_client import QdrantClient

# URL where the Qdrant service is running
url = "http://localhost:6333"

# Name of the collection in Qdrant
collection_name = "medical_dataset"

# Model name and the distance type used for embeddings
embedding_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model_distance = 'Cosine'

# Initialize the Qdrant client
qdrant_client = QdrantClient(url=url, prefer_grpc=False)

# Set the model in the Qdrant client. Ensure this model is properly configured in your Qdrant server.
qdrant_client.set_model(embedding_model_name)

# Initialize the Qdrant Retriever Model
qdrant_retriever_model = QdrantRM(collection_name, qdrant_client, k=3)

# Configure dspy to use the initialized retriever model
dspy.settings.configure(rm=qdrant_retriever_model)

# Setup the Retrieve function with the desired number of top-k passages to retrieve
retrieve = dspy.Retrieve(k=3)

# Define your question
question = "What is Azithromycin?"

# Retrieve the top-K passages for the given question
topK_passages = retrieve(question).passages

# Print the top K passages
print(f"Top {retrieve.k} passages for the question: '{question}'\n")
for idx, passage in enumerate(topK_passages):
    print(f"{idx + 1}]", passage, "\n")
